In [1]:
import pandas as pd
import numpy as np
from src import util
from src.process2004_2017 import get_processed_enade_2014_2017
from src.process2011 import get_processed_enade_2011
from src.process2008 import get_processed_enade_2008
from src import config
from src import subject_manipulation

In [2]:
subject_df = subject_manipulation.get_processed_subject_df()

In [3]:
year = 2008

In [4]:
subject_df = subject_df.loc[subject_df['ano'] == year]
subject_df.head()

,idquestao,ano,curso,prova,tipoquestao,conteudo1,conteudo2,conteudo3
120,1,2008.0,CC,Geral,Objetiva,NaN,NaN,NaN
121,2,2008.0,CC,Geral,Objetiva,NaN,NaN,NaN
122,3,2008.0,CC,Geral,Objetiva,NaN,NaN,NaN
123,4,2008.0,CC,Geral,Objetiva,NaN,NaN,NaN
124,5,2008.0,CC,Geral,Objetiva,NaN,NaN,NaN


In [5]:
subjects = util.get_subjects(subject_df)
subjects

array(['Algoritmos e estruturas de dados',
       'Arquitetura de computadores e sistemas operacionais',
       'Banco de dados', 'Compiladores',
       'Computação gráfica e processamento de imagem',
       'Engenharia de software e interação homem-computador',
       'Fundamentos e técnicas de programação',
       'Inteligência artificial e computacional',
       'Lógica e matemática discreta', 'Probabilidade e estatística',
       'Redes de computadores', 'Sistemas digitais',
       'Teoria da computação', 'Ética, computador e sociedade'],
      dtype=object)

In [6]:
enade_2017_df, original_df = get_processed_enade_2008(f"data/enade/enade{year}/2.DADOS/microdados_enade_{year}.csv")
enade_2017_df.head()

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (49,55,58,68) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


(16, 198)


,QUESTAO_1_NOTA,QUESTAO_2_NOTA,QUESTAO_3_NOTA,QUESTAO_4_NOTA,QUESTAO_5_NOTA,QUESTAO_6_NOTA,QUESTAO_7_NOTA,QUESTAO_8_NOTA,QUESTAO_9_NOTA,QUESTAO_10_NOTA,...,QUESTAO_31_STATUS,QUESTAO_32_STATUS,QUESTAO_33_STATUS,QUESTAO_34_STATUS,QUESTAO_35_STATUS,QUESTAO_36_STATUS,QUESTAO_37_STATUS,QUESTAO_38_STATUS,QUESTAO_39_STATUS,QUESTAO_40_STATUS
295676,0,100,100,0,100,100,0,100,BRANCO,BRANCO,...,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK
295677,0,100,100,0,100,100,100,0,8,8,...,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK
295679,0,100,0,100,100,100,100,100,5,6,...,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK
295681,0,100,100,100,100,0,100,100,5,BRANCO,...,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK
295682,100,100,100,100,100,100,0,100,7,6,...,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK


In [7]:
enade_2017_df["tp_pres"].value_counts()

555    13
222     3
Name: tp_pres, dtype: int64

In [8]:
enade_2017_df["tp_pres"]

295676    555
295677    555
295679    555
295681    555
295682    555
295683    222
295684    555
295697    222
295701    555
295705    555
295712    555
295714    555
295716    555
295717    222
295718    555
295721    555
Name: tp_pres, dtype: int64

In [9]:
new_mapping = enade_2017_df["tp_pres"].map({222:"Ausente", 555:"Presente"}).values
enade_2017_df.loc[:, "tp_pres"] = new_mapping
enade_2017_df["tp_pres"].value_counts()

Presente    13
Ausente      3
Name: tp_pres, dtype: int64

In [10]:
enade_2017_df = enade_2017_df.loc[enade_2017_df["tp_pres"] == "Presente"]

In [11]:
enade_2017_df = util.add_all_score_subjects(enade_2017_df, subject_df, objective=True)

In [12]:
enade_2017_df.head()[[f"QUESTAO_{i}_NOTA" for i in range(11, 21)]]

,QUESTAO_11_NOTA,QUESTAO_12_NOTA,QUESTAO_13_NOTA,QUESTAO_14_NOTA,QUESTAO_15_NOTA,QUESTAO_16_NOTA,QUESTAO_17_NOTA,QUESTAO_18_NOTA,QUESTAO_19_NOTA,QUESTAO_20_NOTA
295676,100,0,100,0,0,100,0,100,0,100
295677,100,0,0,100,100,100,0,100,0,30
295679,0,100,0,0,100,0,0,0,100,27
295681,100,0,0,0,100,100,100,0,0,100
295682,100,0,100,0,100,0,0,100,100,100


In [13]:
enade_2017_df["QUESTAO_22_NOTA"]

295676    100
295677    100
295679    100
295681    100
295682    100
295684    100
295701      0
295705    100
295712      0
295714    100
295716    100
295718    100
295721    100
Name: QUESTAO_22_NOTA, dtype: object

In [14]:
a = enade_2017_df["QUESTAO_1_NOTA"].copy()
a[295686] = 0
a.mean()

21.428571428571427

In [15]:
# medias dos alunos por cada tema, junto com o número de questões
num_questions = []
mean_acertos_by_subject = []
std_acertos_by_subject = []
column_zero_subject = []
column_geq_one_subject = []
column_all_subject = []
num_alunos = enade_2017_df.shape[0]

for subject in subjects:
    subject_questions = util.get_subject_valid_questions(subject, 
                                                                 subject_df,
                                                                 enade_2017_df,
                                                                 just_objective=True)
    num_obj_subject_questions = len(subject_questions)
    
    
    
    mean_acertos_subject = enade_2017_df[f"ACERTOS_OBJ_{subject}"].mean()
    
    std_acertos_subject = enade_2017_df[f"ACERTOS_OBJ_{subject}"].std()
    
    zero_subject_count = enade_2017_df[f"ACERTOS_OBJ_{subject}"].value_counts()

    if 0 in zero_subject_count.index:
        zero_subject = zero_subject_count[0] / num_alunos
    else:
        zero_subject = 0
    
    geq_one_subject_count = (enade_2017_df[f"ACERTOS_OBJ_{subject}"] >= 1).value_counts()
    if True in geq_one_subject_count.index:
        geq_one_subject = geq_one_subject_count[True] / num_alunos
    else:
        geq_one_subject = 0
    
    all_subject_count = (enade_2017_df[f"ACERTOS_OBJ_{subject}"] == num_obj_subject_questions).value_counts()
    if True in all_subject_count.index:
        all_subject = (all_subject_count[True] / num_alunos)
    else:
        all_subject = 0
        
    if num_obj_subject_questions > 0:
        num_questions.append(num_obj_subject_questions)
        mean_acertos_by_subject.append(mean_acertos_subject)
        std_acertos_by_subject.append(std_acertos_subject)
        column_zero_subject.append(zero_subject)
        column_geq_one_subject.append(geq_one_subject)
        column_all_subject.append(all_subject)
    else:
        num_questions.append(0)
        mean_acertos_by_subject.append(None)
        std_acertos_by_subject.append(None)
        column_zero_subject.append(None)
        column_geq_one_subject.append(None)
        column_all_subject.append(None)
        




    
    
subjects_labels = [f"SCORE_OBJ_{x}" for x in subjects]

mean_by_subject = enade_2017_df[subjects_labels].mean().values

data = np.array([mean_by_subject, num_questions]).T

display_df = pd.DataFrame(data=data, index=subjects, 
                          columns=["Nota %", "Nº Questões"])
display_df["Nº Questões"] = display_df["Nº Questões"].astype(int, errors="ignore")
display_df["Média Acertos"] = mean_acertos_by_subject
display_df["Desvio Padrão Acertos"] = std_acertos_by_subject
display_df["% de Zeros"] = column_zero_subject
display_df["% de Zeros"] = display_df["% de Zeros"]*100
display_df["% de Alunos que acertaram pelo menos uma questão"] = column_geq_one_subject
display_df["% de Alunos que acertaram pelo menos uma questão"] = display_df["% de Alunos que acertaram pelo menos uma questão"]*100
display_df["% de Alunos que acertaram todas"] = column_all_subject
display_df["% de Alunos que acertaram todas"] = display_df["% de Alunos que acertaram todas"]*100

display_df = display_df.sort_values(by=["Nota %"]).round(2)
display_df


,Nota %,Nº Questões,Média Acertos,Desvio Padrão Acertos,% de Zeros,% de Alunos que acertaram pelo menos uma questão,% de Alunos que acertaram todas
Probabilidade e estatística,11.54,2,0.23,0.44,76.92,23.08,0.00
Engenharia de software e interação homem-computador,26.92,2,0.54,0.52,46.15,53.85,0.00
Computação gráfica e processamento de imagem,30.77,2,0.62,0.51,38.46,61.54,0.00
Redes de computadores,33.85,5,1.69,1.18,7.69,92.31,0.00
Algoritmos e estruturas de dados,42.31,2,0.85,0.80,38.46,61.54,23.08
Banco de dados,42.31,2,0.85,0.69,30.77,69.23,15.38
Lógica e matemática discreta,42.31,2,0.85,0.69,30.77,69.23,15.38
Sistemas digitais,50.00,2,1.00,0.71,23.08,76.92,23.08
Fundamentos e técnicas de programação,53.85,1,0.54,0.52,46.15,53.85,53.85
Inteligência artificial e computacional,57.69,2,1.15,0.55,7.69,92.31,23.08


In [19]:
original_df[["vt_esc_ofg", "vt_esc_oce"]]

,vt_esc_ofg,vt_esc_oce
295676,DEDCDBBC,CDACDBECCDADEABEAAAEBCCBCEA
295677,DEDEDBDE,CEEEBBBCBDACAAEBAADEECEBACE
295679,EEEADBDC,ECCBBACBADAEABBEAACECCBCADE
295681,EEDADEDC,CEDBBBDBBAACAADEAAAEBDACCBB
295682,CEDADBBC,CDAABAECADADEBBCABAEBCCCAEE
295683,NaN,NaN
295684,EEDADBBE,CDCCCADCABAEACDDADBBBCACABD
295697,NaN,NaN
295701,EBECDDDE,CEDABBCABECDABCEADCDDEBEABE
295705,EEDADBDC,CED.BCDCCDACBBBDADBEAABEDEE


In [16]:
objective_questions = subject_manipulation.get_objective_questions(subject_df)
enade_2017_df_objective = enade_2017_df[[f"QUESTAO_{i}_NOTA" for i in objective_questions]]
num_blank = 0
num_deletion = 0
for column in enade_2017_df_objective.columns:
    if "BRANCO" in enade_2017_df_objective[column].values:
        num_blank += enade_2017_df_objective[column].value_counts()["BRANCO"]
    if "RASURA" in enade_2017_df_objective[column].values:
        num_deletion += enade_2017_df_objective[column].value_counts()["RASURA"]
percentage_blank = num_blank * 100 / np.prod(enade_2017_df_objective.shape)
percentage_blank = round(percentage_blank, 2)



print(f"{num_blank} questão/questões marcada(s) em branco, o que equivale a "
      f"{percentage_blank}% das questões objetivas")
print(f"{num_deletion} questão/questões rasurada(s)")

1 questão/questões marcada(s) em branco, o que equivale a 0.22% das questões objetivas
0 questão/questões rasurada(s)
